In [3]:
import cv2
import numpy as np

In [22]:
import cv2
import numpy as np

# Initialize variables
image_count = 0
previous_shard_id = None

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define range of blue color in HSV
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)

    # Convert the result to grayscale and apply thresholding
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Process each contour
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if w > 20 and h > 20:  # Minimum size threshold for shard
            # Capture image of the detected shard
            if previous_shard_id is None or (x, y, w, h) != previous_shard_id:
                cv2.imwrite(f'shard_{image_count}.jpg', frame[y:y+h, x:x+w])
                print(f'Saved image: shard_{image_count}.jpg')
                image_count += 1
                previous_shard_id = (x, y, w, h)

    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow('Res', res)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
